This notebook was mainly to see whether it is possible to recover the hex customer_id (required for the submission) from the reduced id's introduced by Radek

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

In [3]:
all_transactions = pd.read_parquet('../../../data/transactions_train.parquet')
articles = pd.read_parquet('../../../data/articles.parquet')
customers = pd.read_parquet('../../../data/customers.parquet')

In [16]:
submissions = pd.read_csv('../../../data/sample_submission.csv')
submissions.head(20)

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0706016001 0706016002 0372860001 0610776002 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0706016001 0706016002 0372860001 0610776002 07...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0706016001 0706016002 0372860001 0610776002 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0706016001 0706016002 0372860001 0610776002 07...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0706016001 0706016002 0372860001 0610776002 07...
5,000064249685c11552da43ef22a5030f35a147f723d5b0...,0706016001 0706016002 0372860001 0610776002 07...
6,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,0706016001 0706016002 0372860001 0610776002 07...
7,00007d2de826758b65a93dd24ce629ed66842531df6699...,0706016001 0706016002 0372860001 0610776002 07...
8,00007e8d4e54114b5b2a9b51586325a8d0fa74ea23ef77...,0706016001 0706016002 0372860001 0610776002 07...
9,00008469a21b50b3d147c97135e25b4201a8c58997f787...,0706016001 0706016002 0372860001 0610776002 07...


In [7]:
customers.shape

(1371980, 7)

Sanity check: all customers are actually submitted to the competition

In [10]:
prediction_candidates = pd.read_csv('../../../data/candidates_test.csv.gz')
prediction_candidates

,article_id,customer_id
0,875784002,1.844671e+19
1,751471043,1.844671e+19
2,751471001,1.844671e+19
3,706016001,1.844671e+19
4,919365008,1.844671e+19
...,...,...
16463755,863595006,1.692995e+19
16463756,896152002,1.692995e+19
16463757,448509014,1.692995e+19
16463758,909916001,1.692995e+19


In [12]:
# Set prediction candidates customer id as uint64
prediction_candidates["customer_id"] = prediction_candidates["customer_id"].astype("uint64")
prediction_candidates

,article_id,customer_id
0,875784002,18446705133201055744
1,751471043,18446705133201055744
2,751471001,18446705133201055744
3,706016001,18446705133201055744
4,919365008,18446705133201055744
...,...,...
16463755,863595006,16929951892735598592
16463756,896152002,16929951892735598592
16463757,448509014,16929951892735598592
16463758,909916001,16929951892735598592


In [15]:
# Map customer id to hex using customer_hex_id_to_int
import sys
sys.path.append("../../")
from util import customer_hex_id_to_int

prediction_candidates["customer_id"] = prediction_candidates["customer_id"].apply(customer_hex_id_to_int)
prediction_candidates

AttributeError: 'int' object has no attribute 'str'

In [ ]:
prediction_dict = predictions.groupby("customer_id")["article_id"].apply(list).to_dict()

In [ ]:
output_df = submission_df.copy()

In [ ]:
output_df.prediction = [" ".join(f"0{x}" for x in prediction_dict[customer_id]) for customer_id in customer_hex_id_to_int(submission_df["customer_id"]) if customer_id in prediction_dict]